In [1]:
import pandas as pd
import numpy as np

# Se cargan los datos originales y se muestra el número de variables y registros
df = pd.read_excel('data_original.xlsx', engine='openpyxl')
print('Número de variables:', len(df.columns))
print('Número de registros:', len(df))

Número de variables: 290
Número de registros: 2478


In [2]:
# Función que cuenta el número de variables y registros de un DataFrame

# ENTRADA:
# df: DataFrame con los datos
# SALIDA:
# n_variables: número de variables del DataFrame
# n_registros: número de registros del DataFrame

def contar_variables_registros(df):
    n_variables = len(df.columns)
    n_registros = len(df)
    print('Número de variables:', n_variables)
    print('Número de registros:', n_registros)

# Función que elimina las columnas que superan un porcentaje máximo de valores nulos dado

# ENTRADA:
# df: DataFrame con los datos
# porcentaje: porcentaje máximo de valores nulos que puede tener una columna
# SALIDA:
# df: DataFrame con las columnas que no superan el porcentaje de valores nulos

def quitar_columnas_nulos(df, porcentaje):
    # Se calcula el número máximo de valores nulos que puede tener una columna
    max_nulos = len(df)*porcentaje
    # Se eliminan las columnas que superan el número máximo de valores nulos
    df = df.dropna(axis=1, thresh=max_nulos)
    return df

# Función que elimina las filas que contienen valores nulos en el DataFrame. También devuelve el número de registros que se han eliminado

# ENTRADA:
# df: DataFrame con los datos
# SALIDA:
# df: DataFrame sin las filas que contienen valores nulos
# n_filas: número de filas que se han quitado

def quitar_filas_nulos(df):
    # Se calcula el número de filas que se van a retirar
    n_filas = (df.isnull().sum(axis=1)>0).sum()
    # Se eliminan las filas que contienen valores nulos
    df = df.dropna()
    return df, n_filas

**ANÁLISIS VARIABLES ANTECEDENTES**

In [3]:
# Se define un DataFrame con las columnas de antecedentes y se anteponen una A a las columnas para clasificarlas
df_ant = df[['Edad', 'Sexo', 'Peso', 'Talla', 'HFdeC.isquemica', 'Fumador', 'Diabetes', 'TratamientoDM', 'Hipertension', 'Dislipemia', 'IRC', 'Filtrado Glomerular', 'HBpreKT', 'InfartoAntiguo', 'LocIAMprevio', 'QXcoronariapre', 'AnoQXcoronariapre', 'PuentesQxpre.Tx', 'ACTPprevia', 'AnoACTPprevia', 'TCI.stentpre', 'DA.stentpre', 'Cx.stentpre', 'MOB.stentpre', 'CD.stentpre', 'PL.stentpre', 'IVP.stentpre', 'EnfArtPerif.pre', 'ACVpre', 'CHA2DS2VASC', 'CHADS2', 'Grace', 'PreciseDAPT']].rename(columns={'Edad': 'A_Edad', 'Sexo': 'A_Sexo', 'Peso': 'A_Peso', 'Talla': 'A_Talla', 'HFdeC.isquemica': 'A_HFdeC.isquemica', 'Fumador': 'A_Fumador', 'Diabetes': 'A_Diabetes', 'TratamientoDM': 'A_TratamientoDM', 'Hipertension': 'A_Hipertension', 'Dislipemia': 'A_Dislipemia', 'IRC': 'A_IRC', 'Filtrado Glomerular': 'A_Filtrado Glomerular', 'HBpreKT': 'A_HBpreKT', 'InfartoAntiguo': 'A_InfartoAntiguo', 'LocIAMprevio': 'A_LocIAMprevio', 'QXcoronariapre': 'A_QXcoronariapre', 'AnoQXcoronariapre': 'A_AnoQXcoronariapre', 'PuentesQxpre.Tx': 'A_PuentesQxpre.Tx', 'ACTPprevia': 'A_ACTPprevia', 'AnoACTPprevia': 'A_AnoACTPprevia', 'TCI.stentpre': 'A_TCI.stentpre', 'DA.stentpre': 'A_DA.stentpre', 'Cx.stentpre': 'A_Cx.stentpre', 'MOB.stentpre': 'A_MOB.stentpre', 'CD.stentpre': 'A_CD.stentpre', 'PL.stentpre': 'A_PL.stentpre', 'IVP.stentpre': 'A_IVP.stentpre', 'EnfArtPerif.pre': 'A_EnfArtPerif.pre', 'ACVpre': 'A_ACVpre', 'CHA2DS2VASC': 'A_CHA2DS2VASC', 'CHADS2': 'A_CHADS2', 'Grace': 'A_Grace', 'PreciseDAPT': 'A_PreciseDAPT'})

# También se añaden del DataFrame original los predictandos "Muerte CV" y "Revasc"
df_ant.loc[:, 'MuerteCV'] = df['Muerte CV']
df_ant.loc[:, 'Revasc'] = df['Revasc']

# Se eliminan las columnas que superan el 25% de valores nulos
df_ant = quitar_columnas_nulos(df_ant, 0.25)

# Se discretiza la columna A_Edad usando los terciles
terciles = df_ant['A_Edad'].quantile([0.33, 0.66])
df_ant.loc[:, 'A_Edad'] = pd.cut(df_ant['A_Edad'], 
                                 bins=[df_ant['A_Edad'].min()-1, terciles[0.33], terciles[0.66], df_ant['A_Edad'].max()], 
                                 labels=[0, 1, 2])

# Se completan los valores faltantes de Peso con los valores de Peso.PSS del DataFrame original
df_ant.loc[:, 'A_Peso'] = df_ant['A_Peso'].fillna(df['Peso.PSS'])

# Se completan los valores faltantes de Talla (altura) con los valores de Talla.PSS del DataFrame original
df_ant.loc[:, 'A_Talla'] = df_ant['A_Talla'].fillna(df['Talla.PSS'])

# Se categorizan las variables Peso y Talla (altura), para ello se usa el IMC
df_ant.loc[:, 'A_Peso'] = df_ant['A_Peso'] / (df_ant['A_Talla']/100)**2
bins = [0, 18.5, 25, 30, np.inf]
labels = [1, 2, 3, 4]
df_ant.rename(columns={'A_Peso': 'A_IMC'}, inplace=True)
df_ant.loc[:, 'A_IMC'] = pd.cut(df_ant['A_IMC'], bins=bins, labels=labels)
df_ant.drop('A_Talla', axis=1, inplace=True) # se elimina la columna Talla

# Se completan los valores faltantes de HFdeC.isquemica con -1
df_ant.loc[:, 'A_TratamientoDM'] = df_ant['A_TratamientoDM'].fillna(-1)

# Se discretiza la columna A_HBpreKT usando los terciles
terciles = df_ant['A_HBpreKT'].quantile([0.33, 0.66])
df_ant.loc[:, 'A_HBpreKT'] = pd.cut(df_ant['A_HBpreKT'], 
                                 bins=[df_ant['A_HBpreKT'].min()-1, terciles[0.33], terciles[0.66], df_ant['A_HBpreKT'].max()], 
                                 labels=[0, 1, 2])

df_ant.drop('A_AnoACTPprevia', axis=1, inplace=True) # Fecha

# Discretización de la variable A_Grace usando los terciles
try:
    terciles = df_ant['A_Grace'].quantile([0.33, 0.66])
    df_ant.loc[:, 'A_Grace'] = pd.cut(df_ant['A_Grace'], 
                                 bins=[df_ant['A_Grace'].min()-1, terciles[0.33], terciles[0.66], df_ant['A_Grace'].max()], 
                                 labels=[0, 1, 2])
except KeyError:
    print("A_Grace supera el porcentaje máximo de valores nulos")

# Discretización de la variable A_PreciseDAPT usando los terciles
try:
    terciles = df_ant['A_PreciseDAPT'].quantile([0.33, 0.66])
    df_ant.loc[:, 'A_PreciseDAPT'] = pd.cut(df_ant['A_PreciseDAPT'], 
                                 bins=[df_ant['A_PreciseDAPT'].min()-1, terciles[0.33], terciles[0.66], df_ant['A_PreciseDAPT'].max()], 
                                 labels=[0, 1, 2])
except KeyError:
    print("A_PreciseDAPT supera el porcentaje máximo de valores nulos")

# Se guarda en un df auxiliar las columnas incluyendo valores nulos
df_ant_na = df_ant.copy()

# Se eliminan las filas que contienen valores nulos
df_ant, n_filas = quitar_filas_nulos(df_ant)
print("Número de registros no utilizados (con valores faltantes):", n_filas)

df_ant.to_excel('antecedentes.xlsx', index=False, engine='openpyxl')

contar_variables_registros(df_ant)

A_Grace supera el porcentaje máximo de valores nulos
A_PreciseDAPT supera el porcentaje máximo de valores nulos
Número de registros no utilizados (con valores faltantes): 781
Número de variables: 25
Número de registros: 1697


**ANÁLISIS VARIABLES PROCEDIMIENTO**

In [4]:
# Se define un DataFrame con las columnas de los procedimientos y se anteponen una P a las columnas para clasificarlas
df_proc = df[['Unnamed: 34', 'FE', 'Indicacion', 'Acceso', 'TCI.Actual', 'DA.Actual', 'CX.Actual', 'CD.Actual', 'Mamaria.Actual', 'Safena.Actual', 'TratDecidido', 'FechaQx.Actual', 'Puentes.Actual', 'ACVQX.Actual', 'InfartoQX.Actual', 'MuerteQX.Actual', 'MuerteQXtexto', 'TCI.tratada', 'DA.tratada', 'Cx.tratada', 'CD.tratada', 'Mamaria.tratada', 'Safena.tratada', 'Lesiones Tratadas', 'SINTAX1', 'SINTAX2']].rename(columns={'Unnamed: 34': 'P_Unnamed', 'FE': 'P_FE', 'Indicacion': 'P_Indicacion', 'Acceso': 'P_Acceso', 'TCI.Actual': 'P_TCI.Actual', 'DA.Actual': 'P_DA.Actual', 'CX.Actual': 'P_CX.Actual', 'CD.Actual': 'P_CD.Actual', 'Mamaria.Actual': 'P_Mamaria.Actual', 'Safena.Actual': 'P_Safena.Actual', 'TratDecidido': 'P_TratDecidido', 'FechaQx.Actual': 'P_FechaQx.Actual', 'Puentes.Actual': 'P_Puentes.Actual', 'ACVQX.Actual': 'P_ACVQX.Actual', 'InfartoQX.Actual': 'P_InfartoQX.Actual', 'MuerteQX.Actual': 'P_MuerteQX.Actual', 'MuerteQXtexto': 'P_MuerteQXtexto', 'TCI.tratada': 'P_TCI.tratada', 'DA.tratada': 'P_DA.tratada', 'Cx.tratada': 'P_Cx.tratada', 'CD.tratada': 'P_CD.tratada', 'Mamaria.tratada': 'P_Mamaria.tratada', 'Safena.tratada': 'P_Safena.tratada', 'Lesiones Tratadas': 'P_Lesiones_Tratadas', 'SINTAX1': 'P_SINTAX1', 'SINTAX2': 'P_SINTAX2'})

# También se añaden del DataFrame original los predictandos "Muerte CV" y "Revasc"
df_proc.loc[:, 'MuerteCV'] = df['Muerte CV']
df_proc.loc[:, 'Revasc'] = df['Revasc']

# Se eliminan las columnas que superan el 25% de valores nulos
df_proc = quitar_columnas_nulos(df_proc, 0.25)
df_proc.drop('P_Unnamed', axis=1, inplace=True) # Fecha

# Se discretiza la columna P_FE usando los terciles
terciles = df_proc['P_FE'].quantile([0.33, 0.66])
df_proc.loc[:, 'P_FE'] = pd.cut(df_proc['P_FE'], 
                                 bins=[df_proc['P_FE'].min()-1, terciles[0.33], terciles[0.66], df_proc['P_FE'].max()], 
                                 labels=[0, 1, 2])

# Se guarda en un df auxiliar las columnas incluyendo valores nulos
df_proc_na = df_proc.copy()

# Se eliminan las filas que contienen valores nulos
df_proc, n_filas = quitar_filas_nulos(df_proc)
print("Número de registros no utilizados (con valores faltantes):", n_filas)

contar_variables_registros(df_proc)

df_proc.to_excel('procedimientos.xlsx', index=False, engine='openpyxl')

Número de registros no utilizados (con valores faltantes): 723
Número de variables: 19
Número de registros: 1755


**ANÁLISIS VARIABLES ANTECEDENTES + PROCEDIMIENTO**

In [5]:
# Se forma un DataFrame con las columnas de los antecedentes y los procedimientos
df_ant_proc = pd.concat([df_ant_na.drop(['MuerteCV', 'Revasc'], axis=1), df_proc_na], axis=1)

# Se eliminan las filas que contienen valores nulos
df_ant_proc, n_filas = quitar_filas_nulos(df_ant_proc)
print("Número de registros no utilizados (con valores faltantes):", n_filas)

contar_variables_registros(df_ant_proc)

df_ant_proc.to_excel('antecedentes_procedimientos.xlsx', index=False, engine='openpyxl')

Número de registros no utilizados (con valores faltantes): 877
Número de variables: 42
Número de registros: 1601
